In [ ]:
# 출처 : https://wikidocs.net/233344
# LangChain 설치 및 업데이트
#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [2]:
# 대화형 체인(ConversationChain)
# ChatOpenAI 클래스를 사용하여 대화형 AI 모델의 인스턴스를 생성하고, 이를 통해 사용자와의 대화를 처리할 수 있습니다.
# ConversationChain, ChatOpenAI, ConversationBufferMemory 모듈은 대화 관리 및 메모리 관리에 필요한 기능을 제공합니다.

from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

# 모델 생성
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 체인 생성 = llm 모델 | 대화형버퍼메모리
conversation = ConversationChain(
    llm=llm,
    verbose=False, # 상세한 로깅(false)
    memory=ConversationBufferMemory(memory_key="history"),
)

# 예측
response = conversation.invoke({"input":"2024년 제주도에서 가장유명한것 1가지만 알려줘"})
print(f'*response:\n{response}')
print(f'--'*20)

# 메모리 확인 
history = conversation.memory.load_memory_variables({})["history"]
print(f'*history:\n{history}')
print(f'--'*20)

# 예측
response = conversation.invoke({"input":"가장 멋있는 장소 1곳만 알려줘"})
print(f'*response:\n{response}')
print(f'--'*20)

# 메모리 확인 
history = conversation.memory.load_memory_variables({})["history"]
print(f'*history:\n{history}')
print(f'--'*20)



*response:
{'input': '2024년 제주도에서 가장유명한것 1가지만 알려줘', 'history': '', 'response': '2024년 제주도에서 가장 유명한 것은 제주 올레길입니다. 이 길은 제주도를 한 바퀴 도는 약 422km의 자연 경로로, 산, 바다, 숲, 들판 등 다양한 풍경을 즐길 수 있습니다. 많은 관광객들이 올레길을 걷거나 자전거를 타며 제주도의 아름다운 자연을 감상하고 있습니다.'}
----------------------------------------
*history:
Human: 2024년 제주도에서 가장유명한것 1가지만 알려줘
AI: 2024년 제주도에서 가장 유명한 것은 제주 올레길입니다. 이 길은 제주도를 한 바퀴 도는 약 422km의 자연 경로로, 산, 바다, 숲, 들판 등 다양한 풍경을 즐길 수 있습니다. 많은 관광객들이 올레길을 걷거나 자전거를 타며 제주도의 아름다운 자연을 감상하고 있습니다.
----------------------------------------
*response:
{'input': '가장 멋있는 장소 1곳만 알려줘', 'history': 'Human: 2024년 제주도에서 가장유명한것 1가지만 알려줘\nAI: 2024년 제주도에서 가장 유명한 것은 제주 올레길입니다. 이 길은 제주도를 한 바퀴 도는 약 422km의 자연 경로로, 산, 바다, 숲, 들판 등 다양한 풍경을 즐길 수 있습니다. 많은 관광객들이 올레길을 걷거나 자전거를 타며 제주도의 아름다운 자연을 감상하고 있습니다.', 'response': '가장 멋있는 장소로는 성산일출봉을 추천합니다. 성산일출봉은 제주도에서 일출을 감상할 수 있는 최고의 명소 중 하나로 유명합니다. 화산활화산으로 형성된 이 봉은 독특한 모양과 아름다운 풍경으로 많은 관광객들을 매료시키고 있습니다. 일출을 보기 위해 이곳을 찾는 이들이 많이 있습니다.'}
----------------------------------------
*history:
Human: 2

In [6]:
# 스트리밍
# MyCustomHandler 클래스는 BaseCallbackHandler를 상속받아, 새로운 토큰이 생성될 때마다 이를 출력하는 기능을 정의합니다.
# ChatOpenAI 의 스트리밍 모드를 활성화하고, 사용자 정의 핸들러인 StreamingHandler 인스턴스를 콜백으로 설정합니다.

from langchain_core.callbacks.base import BaseCallbackHandler
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory


# 콜백 MyCustomHandler 생성
class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{token}", end="", flush=True)

# 스트리밍을 활성화하기 위해, ChatModel 생성자에 `streaming=True`를 전달합니다.
# 추가적으로, 사용자 정의 핸들러 리스트를 전달합니다.
stream_llm = ChatOpenAI(
    model = "gpt-3.5-turbo",
    streaming = True,  # stream = True로 설정
    callbacks=[MyCustomHandler()],
)

# 체인 구성
conversation = ConversationChain(
    llm=stream_llm,
    verbose=False,
    memory=ConversationBufferMemory(),
)

# 예측
response=conversation.predict(input="제주도 관광지 1곳만 추천")
print(f'*response:\n{response}')
print(f'--'*20)

response=conversation.predict(input="음식점도 1곳만 추천")
print(f'*response:\n{response}')
print(f'--'*20)

response=conversation.predict(input="숙박업소도 1곳만 추천")
print(f'*response:\n{response}')
print(f'--'*20)


제가 추천하는 제주도 관광지는 우도입니다. 우도는 제주도 동쪽에 위치한 작은 섬으로 유채꽃이 아름다운 우도의 유채꽃 축제는 매년 많은 관광객들이 찾는 곳입니다. 또한 우도의 바닷가에서 바라보는 일출이 아름다워 유명합니다. 우도에는 자전거를 대여하여 섬을 돌아다니는 것도 추천드립니다. 즐거운 여행 되세요!*response:
제가 추천하는 제주도 관광지는 우도입니다. 우도는 제주도 동쪽에 위치한 작은 섬으로 유채꽃이 아름다운 우도의 유채꽃 축제는 매년 많은 관광객들이 찾는 곳입니다. 또한 우도의 바닷가에서 바라보는 일출이 아름다워 유명합니다. 우도에는 자전거를 대여하여 섬을 돌아다니는 것도 추천드립니다. 즐거운 여행 되세요!
----------------------------------------
제가 추천하는 제주도 음식점은 제주 함덕해수욕장 근처에 위치한 "함덕해녀의 집"입니다. 이 음식점은 제주 전통 해물 요리를 맛볼 수 있는 곳으로 유명합니다. 특히 해녀분들이 직접 잡은 해산물을 사용하여 만든 맛있는 해물요리가 많이 판매되고 있습니다. 함덕해수욕장에서 해녀의 집을 방문하시면 제주의 맛을 느낄 수 있을 것입니다. 즐거운 식사 되세요!*response:
제가 추천하는 제주도 음식점은 제주 함덕해수욕장 근처에 위치한 "함덕해녀의 집"입니다. 이 음식점은 제주 전통 해물 요리를 맛볼 수 있는 곳으로 유명합니다. 특히 해녀분들이 직접 잡은 해산물을 사용하여 만든 맛있는 해물요리가 많이 판매되고 있습니다. 함덕해수욕장에서 해녀의 집을 방문하시면 제주의 맛을 느낄 수 있을 것입니다. 즐거운 식사 되세요!
----------------------------------------
제가 추천하는 제주도 숙박업소는 제주시에 위치한 "하얏트 리젠시 제주"입니다. 이 호텔은 제주도의 아름다운 자연 속에 위치하고 있으며, 고급스러운 시설과 서비스로 유명합니다. 또한 호텔 내 레스토랑과 스파 시설도 훌륭하며, 제주의 풍경을 감상하며 편안한 휴식을 취할 수 있는 곳입니다. 하얏트 리젠

In [13]:
# prompt 튜닝
# 이 코드는 langchain.prompts에서 PromptTemplate 클래스를 가져와 사용합니다. 
# PromptTemplate는 템플릿 기반의 프롬프트를 생성하는 데 사용되며, 여기서는 전문가와의 대화를 시뮬레이션하는 템플릿을 정의합니다

from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

template = """
당신은 10년차 엑셀 전문가 입니다. 아래 대화내용을 보고 질문에 대한 적절한 답변을 해주세요

# 대화내용
{chat_history}
---
Q : {question}
A :
"""

prompt = PromptTemplate.from_template(template)

#prompt.parial(chat_history="여행을 잘 하는 방법에 대해 알려주세요")

# ConversationChain 클래스의 인스턴스를 생성합니다. 
# 여기서 llm은 언어 모델을, prompt는 대화의 시작점을, 
# memory는 대화의 맥락을 저장하는 메모리 버퍼를, input_key는 사용자 입력을 받는 키를, 
# 그리고 verbose는 상세한 로깅을 활성화할지 여부를 지정합니다. 
# 특히, memory는 ConversationBufferMemory 클래스를 사용하여 memory_key를 통해 어떤 정보를 기억할지 결정합니다.

# 모델 생성
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 체인 생성
conversation = ConversationChain(
    llm=llm,
    prompt=prompt, # 튜닝한 프롬프트를 입력
    memory=ConversationBufferMemory(memory_key="chat_history"), 
    input_key="question",
)

# 예측
response=conversation.predict(question="엑셀에서 VLOOKUP 함수는 무엇인가요? 간단하게 설명해주세요")
print(f'*response:\n{response}')
print(f'--'*20)

# 예측
response=conversation.predict(question="예제를 보여주세요")
print(f'*response:\n{response}')
print(f'--'*20)

# 메모리 history 출력 해봄 = >chat_history 로 지정해야 함.
print('*history')
print(conversation.memory.load_memory_variables({})["chat_history"]) 

*response:
VLOOKUP 함수는 엑셀에서 사용되는 함수로, 특정 값을 검색하고 해당 값을 포함하는 행의 다른 열의 값을 반환하는 함수입니다. 이를 통해 특정 값을 기준으로 다른 데이터를 찾거나 매칭시킬 수 있습니다.
----------------------------------------
*response:
Sure! 예를 들어, A 열에 학생들의 이름이 있고 B 열에 학생들의 성적이 있는 경우, VLOOKUP 함수를 사용하여 특정 학생의 성적을 검색할 수 있습니다. 예를 들어, =VLOOKUP("홍길동", A1:B10, 2, FALSE)와 같이 사용하면 "홍길동"이라는 학생의 성적을 찾을 수 있습니다.
----------------------------------------
*history
Human: 엑셀에서 VLOOKUP 함수는 무엇인가요? 간단하게 설명해주세요
AI: VLOOKUP 함수는 엑셀에서 사용되는 함수로, 특정 값을 검색하고 해당 값을 포함하는 행의 다른 열의 값을 반환하는 함수입니다. 이를 통해 특정 값을 기준으로 다른 데이터를 찾거나 매칭시킬 수 있습니다.
Human: 예제를 보여주세요
AI: Sure! 예를 들어, A 열에 학생들의 이름이 있고 B 열에 학생들의 성적이 있는 경우, VLOOKUP 함수를 사용하여 특정 학생의 성적을 검색할 수 있습니다. 예를 들어, =VLOOKUP("홍길동", A1:B10, 2, FALSE)와 같이 사용하면 "홍길동"이라는 학생의 성적을 찾을 수 있습니다.


In [14]:
# Human Prefix & AI Prefix 변경
# history 출력해 보면 Human, AI 로 보이는데, 여기서는 이를 변경하는 방법을 설명한다.

from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

template = """
당신은 10년차 엑셀 전문가 입니다. 아래 대화내용을 보고 질문에 대한 적절한 답변을 해주세요

# 대화내용
{context}
---
Q : {question}
A :
"""

prompt = PromptTemplate.from_template(template)

# ConversationBufferMemory 클래스는 대화의 맥락을 관리하는 데 사용됩니다. 
# 이 클래스의 인스턴스를 생성할 때, memory_key는 대화의 맥락을 저장하는 키로 사용되며, 
# human_prefix와 ai_prefix는 각각 사람과 AI가 말하는 부분에 붙는 접두사로 설정됩니다. 
# 이를 통해 대화 내용을 더 명확하게 구분할 수 있습니다.

memory=ConversationBufferMemory(
    memory_key = "context", # 대화 내용을 저장하는 키
    human_prefix = "Q", # *사람에 대한 prefix
    ai_prefix = "A", # *AI가 말하는 부분에 대한 prefix
)

# 모델 생성
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 체인 생성
conversation = ConversationChain(
    llm=llm,
    prompt=prompt, # 튜닝한 프롬프트를 입력
    memory=memory, # 위에서 지정한 ConversationBufferMemory 지정
    input_key="question",
    verbose=False,
)

# 예측
response=conversation.predict(question="엑셀에서 VLOOKUP 함수는 무엇인가요? 간단하게 설명해주세요")
print(f'*response:\n{response}')
print(f'--'*20)

# 예측
response=conversation.predict(question="예제를 보여주세요")
print(f'*response:\n{response}')
print(f'--'*20)

# 메모리 context 출력 해봄 = >context 로 지정해야 함.
print('*context')
print(conversation.memory.load_memory_variables({})["context"]) 

*response:
VLOOKUP 함수는 주어진 값과 일치하는 값을 다른 범위에서 찾아 반환하는 엑셀 함수입니다. 주로 데이터베이스에서 원하는 정보를 검색할 때 사용됩니다.
----------------------------------------
*response:
예를 들어, A 열에 과일 이름이 나열되어 있고 B 열에는 해당 과일의 가격이 나열되어 있다고 가정해봅시다. VLOOKUP 함수를 사용하여 "사과"이라는 과일의 가격을 찾고 싶을 때, 다음과 같이 사용할 수 있습니다.

=VLOOKUP("사과", A1:B10, 2, FALSE)

위의 식은 A1부터 B10 범위에서 "사과"라는 값을 찾고, 해당 값이 위치한 열에서 오른쪽으로 2번째 열에 있는 값을 반환합니다. FALSE 옵션은 정확하게 일치하는 값을 찾기 위해 사용됩니다.
----------------------------------------
*context
Q: 엑셀에서 VLOOKUP 함수는 무엇인가요? 간단하게 설명해주세요
A: VLOOKUP 함수는 주어진 값과 일치하는 값을 다른 범위에서 찾아 반환하는 엑셀 함수입니다. 주로 데이터베이스에서 원하는 정보를 검색할 때 사용됩니다.
Q: 예제를 보여주세요
A: 예를 들어, A 열에 과일 이름이 나열되어 있고 B 열에는 해당 과일의 가격이 나열되어 있다고 가정해봅시다. VLOOKUP 함수를 사용하여 "사과"이라는 과일의 가격을 찾고 싶을 때, 다음과 같이 사용할 수 있습니다.

=VLOOKUP("사과", A1:B10, 2, FALSE)

위의 식은 A1부터 B10 범위에서 "사과"라는 값을 찾고, 해당 값이 위치한 열에서 오른쪽으로 2번째 열에 있는 값을 반환합니다. FALSE 옵션은 정확하게 일치하는 값을 찾기 위해 사용됩니다.
